# 帕金森病进展预测

## 介绍
### 数据集描述
+ 本次竞赛的目标是使用蛋白质丰度数据预测帕金森病（PD）的病程。参与PD的全套蛋白质仍然是一个悬而未决的研究问题，任何具有预测价值的蛋白质都可能值得进一步研究。该数据集的核心包括蛋白质丰度值，该值来自从数百名患者收集的脑脊液（CSF）样本的质谱读数。每位患者在多年的时间里提供了几个样本，同时还对PD的严重程度进行了评估。
+ 这是一个时间序列代码竞赛：您将接收测试集数据并使用Kaggle的时间序列API进行预测。
### 文件
+ train_peptides.csv肽水平的质谱数据。肽是蛋白质的组成亚基。
  + visit_id- 访问的ID代码。
  + visit_month- 就诊月份，相对于患者第一次就诊。
  + patient_id- 患者的ID码。
  + UniProt - 相关蛋白质的 [UniProt ID](https://www.uniprot.org/id-mapping)  代码。每种蛋白质通常有几种肽。
  + Peptide- 肽中包含的氨基酸序列。有关 [相关代码](./Analysis/Parkinson's-Disease-Progression-Prediction/id.csv) 。表中可能不包含一些罕见的注释。测试集可能包括在训练集中未找到的肽。**(unimod_4)羧基酰胺甲基化**
  + PeptideAbundance- 样品中氨基酸的频率。
  
>
+ train_proteins.csv从肽水平数据汇总的蛋白表达频率。
  + visit_id- 访问的ID代码。
  + visit_month- 就诊月份，相对于患者第一次就诊。
  + patient_id- 患者的ID码。
  + UniProt - 相关蛋白质的 [UniProt ID](https://www.uniprot.org/id-mapping) 代码。每种蛋白质通常有几种肽。测试集可能包括在训练集中未发现的蛋白质。
  + NPX- 标准化蛋白表达。蛋白质在样品中出现的频率。可能与组分肽没有1：1的关系，因为某些蛋白质含有给定肽的重复拷贝。
  
>
+ train_clinical_data.csv
  + visit_id- 访问的ID代码。
  + visit_month- 就诊月份，相对于患者第一次就诊。
  + patient_id- 患者的ID码。
  + updrs_[1-4]- 统一帕金森病评定量表N部分的患者评分。数字越高表示症状越严重。每个小节涵盖一类不同的症状，例如第 1 部分的情绪和行为以及第 3 部分的运动功能。
  
>
+ upd23b_clinical_state_on_medication- 患者在UPDRS评估期间是否服用左旋多巴等药物。预计主要影响第 3 部分（运动功能）的分数。这些药物消退得相当快（大约一天），因此患者通常在一个月内进行两次运动功能检查，包括有药和不服药。
  
>
+ supplemental_clinical_data.csv没有任何相关脑脊液样本的临床记录。该数据旨在提供有关帕金森病典型进展的其他背景信息。使用与train_clinical_data.csv相同的列。
  
>
+ example_test_files/旨在说明 API 如何运行的数据。包括 API 提供的相同列（即没有 updrs 列）。
  
>
+ amp_pd_peptide/启用 API 的文件。预计 API 将在 5 分钟内提供所有数据（少于 1，000 名额外患者），并保留少于 0.5 GB 的内存。此处提供了有关 API 交付内容的简要演示。
  
>
+ public_timeseries_testing_util.py一个可选文件，旨在更轻松地运行自定义脱机 API 测试。有关详细信息，请参阅脚本的文档字符串。

>

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 分析数据

## train 数据集

### 读取数据

In [175]:
train_proteins_data=pd.read_csv('data/amp-parkinsons-disease-progression-prediction/train_proteins.csv',sep=',')
# 缺失值统计
print(train_proteins_data.isnull().sum())
train_proteins_data

visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [176]:
train_peptides_data=pd.read_csv('data/amp-parkinsons-disease-progression-prediction/train_peptides.csv',sep=',')
# 缺失值统计
print(train_peptides_data.isnull().sum())
train_peptides_data

visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [177]:
train_clinical_data=pd.read_csv('data/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv',sep=',')
# 缺失值统计
print(train_clinical_data.isnull().sum())
train_clinical_data

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0,Off
2611,65043_54,65043,54,4.0,8.0,11.0,1.0,Off
2612,65043_60,65043,60,6.0,6.0,16.0,1.0,Off
2613,65043_72,65043,72,3.0,9.0,14.0,1.0,Off


### 特征比较

#### 分类统计

In [178]:
#汇总病人
group_by_patient_id=train_clinical_data.groupby('patient_id').count()
patient_id=train_clinical_data.groupby('patient_id').count().index
max_visit_time=train_peptides_data['visit_month'].max()

print(' 病人总人数',len(group_by_patient_id),'\n','时间序列计数最大值',max_visit_time)

print('病人序列',patient_id)

 病人总人数 248 
 时间序列计数最大值 108
病人序列 Int64Index([   55,   942,  1517,  1923,  2660,  3636,  3863,  4161,  4172,
             4923,
            ...
            62329, 62437, 62723, 62732, 62792, 63875, 63889, 64669, 64674,
            65043],
           dtype='int64', name='patient_id', length=248)


In [179]:
#病人序列patient_id转字典
patient_id_dict={}
for i in range(len(patient_id)):
    patient_id_dict[str(patient_id[i])]=i

#upd23b_clinical_state_on_medication转为0/1,nan视为0
upd23b_clinical_state_on_medication=pd.Categorical(train_clinical_data['upd23b_clinical_state_on_medication'].fillna('Off')).codes

#构建时间*样本-得分矩阵updrs
Time_sample_updrs=np.zeros((4,284,109))

#时间*样本-给药矩阵
Time_sample_clinical_state_on_medication=np.zeros((1,284,109),dtype=bool)

for idx in range(len(train_clinical_data['visit_id'])):
    #updrs_1	
    Time_sample_updrs[0][
        patient_id_dict[str(train_clinical_data['patient_id'][idx])]][train_clinical_data['visit_month'][idx]]=train_clinical_data['updrs_1'][idx]
    #updrs_2	
    Time_sample_updrs[1][
        patient_id_dict[str(train_clinical_data['patient_id'][idx])]][train_clinical_data['visit_month'][idx]]=train_clinical_data['updrs_2'][idx]
    #updrs_3	
    Time_sample_updrs[2][
        patient_id_dict[str(train_clinical_data['patient_id'][idx])]][train_clinical_data['visit_month'][idx]]=train_clinical_data['updrs_3'][idx]
    #updrs_4	
    Time_sample_updrs[3][
        patient_id_dict[str(train_clinical_data['patient_id'][idx])]][train_clinical_data['visit_month'][idx]]=train_clinical_data['updrs_4'][idx]
    #upd23b_clinical_state_on_medication
    Time_sample_clinical_state_on_medication[0][
        patient_id_dict[str(train_clinical_data['patient_id'][idx])]][
        train_clinical_data['visit_month'][idx]]=upd23b_clinical_state_on_medication[idx]

#print(Time_sample_updrs,Time_sample_clinical_state_on_medication)

#### 绘图分析

In [ ]:
#绘制按样本分类时间-得分曲线
for updrs in range(4):
    plt.figure(figsize=(50,50))
    for idx_y in range(len(Time_sample_updrs[updrs])):
    
        x=[]
        y=[]

        for i in range(len(Time_sample_updrs[updrs][idx_y])):
            if Time_sample_updrs[updrs][idx_y][i] != 0:
                x.append(i)
                y.append(Time_sample_updrs[updrs][idx_y][i])

        if(len(x)>1 and len(y)>1):
            plt.plot(x,y)
    plt.show()

## 数据处理

## 构建模型

## 训练

## 预测